In [ ]:
import logging, os
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"

In [ ]:
from sklearn.metrics import classification_report, confusion_matrix


In [1]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, BatchNormalization
from matplotlib import pyplot as plt
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix


2024-02-24 05:52:30.659695: I tensorflow/stream_executor/platform/default/dso_loader.cc:48] Successfully opened dynamic library libcudart.so.10.1


In [ ]:
print(tf.config.list_physical_devices('GPU'))
print("TensorFlow version:", tf.__version__)

In [ ]:
data_dir = 'asl_alphabet_train'

list_ds = tf.data.Dataset.list_files(str(data_dir + '/*/*.jpg'), shuffle=False)
image_count = len(list_ds)
list_ds = list_ds.shuffle(image_count, reshuffle_each_iteration=True)

class_names = np.array(sorted([item.name for item in os.scandir(data_dir)]))
num_classes = len(class_names)

In [ ]:
print(f"The total number of images in the dataset is {image_count}.")
print(f"The number of different classes is {num_classes}.")

In [ ]:
batch_size = 10
img_height = 200
img_width = 200
channel = 3

In [ ]:
# # Splitting into training and validation sets
val_size = int(image_count * 0.2)
train_ds = list_ds.skip(val_size)
val_ds = list_ds.take(val_size)

In [ ]:
def preprocess_image(file_path):
    # Load the image
    img = tf.io.read_file(file_path)
    img = tf.image.decode_jpeg(img, channels=channel)
    img = tf.image.resize(img, [img_height, img_width])

    # Data augmentation
    img = tf.image.random_flip_left_right(img)  # Randomly flip the image left to right
    img = tf.image.random_flip_up_down(img)  # Randomly flip the image upside down
    img = tf.image.random_brightness(img, max_delta=0.1)  # Randomly adjust the brightness
    img = tf.image.random_contrast(img, lower=0.8, upper=1.2)  # Randomly adjust the contrast

    # # Rescale pixel values
    img /= 255.0
    return img

In [ ]:
def get_label(file_path):
    parts = tf.strings.split(file_path, os.path.sep)
    return parts[-2] == class_names

In [ ]:
train_ds = train_ds.map(lambda x: (preprocess_image(x), get_label(x)),
                        num_parallel_calls=tf.data.experimental.AUTOTUNE)
train_ds = train_ds.batch(batch_size)
train_ds = train_ds.prefetch(tf.data.experimental.AUTOTUNE)

val_ds = val_ds.map(lambda x: (preprocess_image(x), get_label(x)),
                    num_parallel_calls=tf.data.experimental.AUTOTUNE)
val_ds = val_ds.batch(batch_size)
val_ds = val_ds.prefetch(tf.data.experimental.AUTOTUNE)

In [ ]:
image, label = next(iter(val_ds.take(2)))

# Access the label of the first example
first_label = image[0]

plt.imshow(first_label, cmap="gray")
plt.axis('off')
plt.show()

In [ ]:
model = Sequential()

# Add the layers to the model with the specified input shape
model.add(Conv2D(filters=96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(img_height, img_width, channel), padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Conv2D(filters=256, kernel_size=(5, 5), activation='relu', padding='same'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Conv2D(filters=384, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(Conv2D(filters=384, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(Conv2D(filters=256, kernel_size=(3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))

model.add(Flatten())
model.add(Dense(4096, activation='relu'))
model.add(Dense(4096, activation='relu'))

model.add(Dense(num_classes, activation='softmax'))

model.compile(
    optimizer='sgd',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

In [ ]:
# Model Summary
model.summary()

In [ ]:
# Define early stopping criteria
early_stopping = EarlyStopping(
    monitor='val_accuracy',  # Metric to monitor
    patience=3,           # Number of epochs to wait for improvement
    restore_best_weights=True  # Restore model weights to those that achieved the best value of the monitored metric
)

# Train the model with early stopping
history = model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=10,
    verbose=1,
    callbacks=[early_stopping]  # Include early stopping callback
)

In [ ]:
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label = 'val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.ylim([0, 1])
plt.legend(loc='lower right')
plt.show()

plt.plot(history.history['loss'], label='loss')
plt.plot(history.history['val_loss'], label = 'val_loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.ylim([0, max(np.max(history.history['loss']), np.max(history.history['val_loss']))])
plt.legend(loc='upper right')
plt.show()

In [ ]:
evaluation = model.evaluate(val_ds)

# Print the evaluation results
print("Validation Loss:", evaluation[0])
print("Validation Accuracy:", evaluation[1])

In [ ]:
# Initialize empty lists to store true labels and predictions
true_labels = []
predictions = []

# Iterate over the test dataset to extract true labels and predictions
for images, labels in test_ds:
    true_labels.extend(np.argmax(labels, axis=1))  # Convert one-hot encoded labels to categorical labels
    predictions.extend(np.argmax(model.predict(images), axis=1))  # Get predictions

# Convert lists to numpy arrays
true_labels = np.array(true_labels)
predictions = np.array(predictions)

# Print classification report
print(classification_report(true_labels, predictions))

# Plot confusion matrix
plt.figure(figsize=(12, 12))
sns.heatmap(confusion_matrix(true_labels, predictions), annot=True, fmt="d", cmap="Blues")
plt.show()

In [ ]:
# model.save("my_model.h5")